In [1]:
import pandas as pd
import numpy as np
from ta.momentum import rsi
from ta.utils import dropna
import datetime
from api_helper import ShoonyaApiPy
import logging
import sched, time

In [2]:
#enable dbug to see request and responses
logging.basicConfig(level=logging.DEBUG)

#start of our program
api = ShoonyaApiPy()

In [3]:
def loginme():
    #credentials
    uid    = 'FA54439'
    pwd     = 'Gurdeep#7966'
    factor2 = '23031990'
    vc      = 'FA54439_U'
    apikey  = 'a4c5620bebfb7be65eb2821deab0eb62'
    imei    = 'GuruPC'

    #make the api call
    login = api.login(userid=uid, password=pwd, twoFA=factor2, vendor_code=vc, api_secret=apikey, imei=imei)
    if login['actid']=='FA54439':
        print('Login Success!!')
    else:
        print('Logout Failed!!')
    return True


In [4]:
def logoutme():
    logout = api.logout()
    if logout['stat']=='Ok':
        print('Logout Success!!')
    return True

In [5]:
def Supertrend(df, atr_period, multiplier):
    
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    # calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # df['atr'] = df['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    return pd.DataFrame({
        'Supertrend': supertrend,
        'Final Lowerband': final_lowerband,
        'Final Upperband': final_upperband
    }, index=df.index)

In [6]:
def searchsymbol(exch,scr):   
    exch  = exch #NSE,NFO,MCX etc
    query = scr #'BANKNIFTY 30DEC CE' # multiple criteria to narrow results 
    # ret = api.searchscrip(exchange=exch, searchtext=query)
    mytoken=api.searchscrip(exchange=exch, searchtext=query)['values'][0]['token']
    return mytoken

In [7]:
# api.get_security_info(exchange='NSE', token=niftytoken)

In [8]:
def getdataop(token,time):    
    lastBusDay = datetime.datetime.today()-datetime.timedelta(days=5)
    lastBusDay = lastBusDay.replace(hour=0, minute=0, second=0, microsecond=0)
    ret = api.get_time_price_series(exchange='NFO', token=token, starttime=lastBusDay.timestamp(), interval=time)
    df=pd.DataFrame(ret)
    df=df.iloc[::-1]
    df['time']=pd.to_datetime(df['time'].astype(str), format='%d-%m-%Y %H:%M:%S')
    df.set_index('time',inplace=True)
    df=df.iloc[:,2:]
    df=df.astype(float)
    # return only last 300 bars
    df=df.iloc[-300:]
    return df

In [9]:
def getdataeq(token,time):    
    lastBusDay = datetime.datetime.today()-datetime.timedelta(days=5)
    lastBusDay = lastBusDay.replace(hour=0, minute=0, second=0, microsecond=0)
    ret = api.get_time_price_series(exchange='NSE', token=token, starttime=lastBusDay.timestamp(), interval=time)
    df=pd.DataFrame(ret)
    df=df.iloc[::-1]
    df['time']=pd.to_datetime(df['time'].astype(str), format='%d-%m-%Y %H:%M:%S')
    df.set_index('time',inplace=True)
    df=df.iloc[:,2:]
    df=df.astype(float)
    # return only last 300 bars
    df=df.iloc[-300:]
    return df

In [66]:
###Get ITM CE and PE Strikes
def get_ITM_strike(df):
    spot=df.intc.iloc[-1:][0]
    atm=50 * round(int(spot)/50)
    itm_ce=atm-50
    itm_pe=atm+50
    # Returning spot as float and both strikes as int
    return spot,itm_ce,itm_pe

In [68]:
####Get ITM CE Token for NIFTY Only
def get_option_token_ce(expiry,strike):
    cscr=f'NIFTY{expiry}22C{strike}'
    tokence=searchsymbol(exch='NFO',scr=cscr)
    return cscr,tokence

In [67]:
####Get ITM PE Token for NIFTY Only
def get_option_token_pe(expiry, strike):
    pscr=f'NIFTY{expiry}22P{strike}'
    tokenpe=searchsymbol(exch='NFO',scr=pscr)
    return pscr,tokenpe

In [69]:
def resampler(df,time):
    newdf=pd.DataFrame()
    newdf['Open']=df.into.resample(time).first()
    newdf['Close']=df.intc.resample(time).last()
    newdf['High']=df.inth.resample(time).max()
    newdf['Low']=df.intl.resample(time).min()
    newdf['Vwap']=df.intvwap.resample(time).last()
    newdf['Volume']=df.intv.resample(time).sum()
    newdf['OI']=df.intoi.resample(time).sum()
    ## Folloing drop will come in play when using time frame like 10 min or 20 min etc .beacuse these time frame starts from 9:10 am or 9:00 am 
    newdf.dropna(axis=0,inplace=True)
    return newdf

In [14]:
def predf(data):
    
    ###Base Time Frame########
    datadf=data.copy()
    
    datadf=resampler(datadf,'5min')
    supertrendBTF=Supertrend(datadf,10,1)
    supertrendBTF['rsi']=rsi(datadf.Close,window=8,fillna=True)
    supertrendBTF['rsima']=supertrendBTF['rsi'].rolling(13).mean()
    supertrendBTF.reset_index(inplace=True)

    ###High Time Frame#########
    datadf=data.copy()
    
    datadf=resampler(datadf,'15min')
    supertrendHTF=Supertrend(datadf,10,0.8)
    supertrendHTF['rsi']=rsi(datadf.Close,window=8,fillna=True)
    supertrendHTF['rsima']=supertrendHTF['rsi'].rolling(13).mean()
    supertrendHTF.reset_index(inplace=True)
    
    #######Merge Base and HTF indicators###
    st=pd.merge(supertrendBTF,supertrendHTF,on='time',how='left')
    st=st.replace(to_replace=np.nan,method='ffill')
    st=st.set_index('time')
    
    #########Join everything in single df#######
    datadf=data.copy()
    
    datadf=resampler(datadf,'5min')
    datadf=datadf.join(st)
    
    
    return datadf

In [15]:
loginme()

Login Success!!


True

In [16]:
####Position Book######
def check_position(ticker):
    pos=False
    posret = api.get_positions()
    positionsdf=pd.DataFrame(posret)
    in_positions=positionsdf.loc[positionsdf.tsym==ticker]
    if not in_positions.empty:
        if int(in_positions['netqty'].iloc[0])==0:
            pos=False
            print('not in position')
        else:
            pos=True
            print(' In position')
    else:
        pos=False
    return pos



In [17]:
####Order Book########
def check_orderbook(ticker):
    orderret = api.get_order_book()
    orderdf=pd.DataFrame(orderret)
    in_orderbook=orderdf.loc[orderdf.tsym==ticker]['status']
    if  not in_orderbook.empty:
        ## Always check 1st index beacuse this holds the latest position
        orderstatus=in_orderbook.iloc[0]
        if orderstatus=='COMPLETE':
            print (' order filled!! check position book')
        elif orderstatus=='OPEN':
            print ('previous order still open')
        elif orderstatus=='CANCELED':
            print ('order is canceled')
        elif orderstatus=='REJECTED':
            print ('order is REJECTED')
        else: 
            orderstatus=='UNKNOWN'
    else:
        orderstatus='UNKNOWN'
        print('No order found')
    return orderstatus

In [18]:
def check_tradebook(ticker):
    tradebookret = api.get_trade_book()
    tradebookdf=pd.DataFrame(tradebookret)
    in_tradebook=tradebookdf.loc[tradebookdf.tsym==ticker]
    if check_orderbook(ticker)=='COMPLETE':
        if not in_tradebook.empty:
            fill_price=float(in_tradebook['flprc'].iloc[0])  
    else:
        print('Order not yet Filled')
    return fill_price
    

In [60]:
def exe_limt_order(order_type,product_type,exchange,ticker,quantity,ltp,triger_ltp,remarks):
    resp=api.place_order(buy_or_sell=order_type, product_type=product_type,exchange=exchange, tradingsymbol=ticker, 
                    quantity=quantity, discloseqty=0,price_type='LMT', price=ltp, trigger_price=None,
                    retention='DAY', remarks=remarks)
    return resp

In [63]:
def exe_market_order(order_type,product_type,exchange,ticker,quantity,remarks):
    resp=api.place_order(buy_or_sell=order_type, product_type=product_type,exchange=exchange, tradingsymbol=ticker, 
                    quantity=quantity, discloseqty=0,price_type='MKT', price=0, trigger_price=None,
                    retention='DAY', remarks=remarks)
    return resp

In [19]:
# #Flow--
# '''
# --Place Order--
# 1.Check if postion is not open already .Call check_position and it shoud return False
# 2.if position is false ,check if order is not in open state .
#     -if no trade done for the symbol from starting of day , it will return order as UNKNOWN.
#     -if already done trading on that symbol from day starting, it will return Complete or Open or Cacneled ,Rejected 
#      based on last entry in order book.
#     - So if check_orderbook return  COMPLETE  or UNKNOWN then we are good to place order.
# 3. Place order.
# 4. check orderbook,if return COMPLETE. check trade book and get price value at which order is placed.


# ---Exit Order
# 1. Check if postion is not open already .Call check_position and it shoud return True.
# 2. Punch exit order .
# 3. Check order and check if order complete.if still open exit at market order.
# 4.check trade book to get exit price.
# '''


In [20]:
#This is Scheduler.This will start main program at exectly hr:min:00 and will wait for whatever time you give
##Note that even you run the loop for less then 1min time ,it will get data only after next 1 min .
# sec=True
# while sec==True:
#     print(float(str(datetime.datetime.now().time()).split(':')[2]))
#     starttime = time.time()
#     time.sleep(1.0 - ((time.time() - starttime) % 60.0))
#     if float(str(datetime.datetime.now().time()).split(':')[2])>58.0:
#         sec=False

# while True:
#     starttime = time.time()
#     time.sleep(62.0 - ((time.time() - starttime) % 60.0))

In [73]:
def mainloop():
    ## Below variables to store script name and token in case that script in buy position.
    ce_buy_script=''
    pe_buy_script=''
    ce_buy_token=''
    pe_buy_token=''
    ##Loop will come here----
    ## Folling loop is just to make sure Actual loop start from 1 sec of a min
    sec=True
    while sec==True:
        print(float(str(datetime.datetime.now().time()).split(':')[2]))
        starttime = time.time()
        time.sleep(1.0 - ((time.time() - starttime) % 60.0))
        if float(str(datetime.datetime.now().time()).split(':')[2])>58.0:
            sec=False
    ## Folling is the actual loop that will run in every 1 min ,will make it 2-4 min in future.
    now=datetime.datetime.now().time()
    while (now >= datetime.time(9,17)) and (now <= datetime.time(15,10)):
        starttime = time.time()
        time.sleep(60.0 - ((time.time() - starttime) % 60.0))
    #------------------------

        #Start Only if time > 9:15 am
        # now=datetime.datetime.now().time()
        # if now >= datetime.time(9,20):
        ## Get nifty token and Data to calculate ITM CE and PE Strikes
        mytoken=searchsymbol('NSE','Nifty 50')
        nifty=getdataeq(mytoken,1)
        #following will return spot price ,itm strike for both ce and pe
        spot,itm_ce,itm_pe=get_ITM_strike(nifty)
        print(f' Spot: {spot}, ITM Call Strike: {itm_ce}, ITM PUT Strike: {itm_pe}')

        #### This if condition to check if already in trade for PE/CE if yes fetch data for same Strike(token) instead new ITM
        if not ce_buy_token:
            ##Get NEW ITM CE token and script name and download the data for that token.
            cscr,tokence=get_option_token_ce('17MAR',itm_ce)
            cedf=getdataop(token=tokence,time=1)
            ## This will check if any exising position for this script.Will return True if alraedy in position else false
            ceposition = check_position(cscr)
            ## This will return Unkown or Completed or Open or Rejected 
            order_status_ce=check_orderbook(cscr)
        else:
            cedf=getdataop(token=ce_buy_token,time=1)
            ceposition = check_position(ce_buy_script)
            order_status_ce=check_orderbook(ce_buy_script)
        if not pe_buy_token:
            ##Get NEW ITM PE token and script name and download the data for that token.
            pscr,tokenpe=get_option_token_pe('17MAR',itm_pe)
            pedf=getdataop(token=tokenpe,time=1)
            peposition = check_position(pscr)
            order_status_pe=check_orderbook(pscr)
        else:
            pedf=getdataop(token=pe_buy_token,time=1)
            peposition = check_position(pe_buy_script)
            order_status_pe=check_orderbook(pe_buy_script)


        ##Preparing the Data before triggring actual Buy Sell Conditions.Here it will resapme data into 5 and 15 min.
        ## Then it will calculate ST and RSI on both time frames. then it will merge all togeter and will craete single
        ## df with 5 min timestamp.
        cedf=predf(cedf)
        pedf=predf(pedf)

        # # checking if for ticker position is True or False
        # ceposition = check_position(cscr)
        # peposition = check_position(pscr)

        if (ceposition==False):
            #Before placing order checking if no other order for same script in open state.
            
            if ((order_status_ce=='COMPLETE') | (order_status_ce=='UNKNOWN')) & (now<datetime.time(15,0)):##Here Check if you already have position in ITM Call Option
                if (cedf['Supertrend_y'].iloc[-1]==True) & (cedf['Supertrend_x'].iloc[-1]==True)& (cedf['rsi_y'].iloc[-1]>50) & (cedf['rsi_y'].iloc[-1]>cedf['rsima_y'].iloc[-1]):
                    current_ce_price=cedf.Close[-1]
                    ## Place Limit/Market order here. if limit order not failed you will end up add multilple orders.so need to careful here                
                    resp_ce=exe_limt_order('B','M','NFO',cscr,50,current_ce_price,current_ce_price,'CE Limit order Buy Triggred')
                    ## Folloing line will triger market Buy order 
                    #resp_ce=exe_market_order('B','M','NFO',cscr,50,'Market order buy Triggred')
                    ## Check if oder is completed,wait for 1 sec before checking
                    time.sleep(2)
                    order_status=check_orderbook(cscr)
                    if order_status=='COMPLETE':
                        #get buy price from trade book.Folling function will return last price for the script
                        cbp=check_tradebook(cscr)
                        ce_buy_script=cscr
                        ce_buy_token=tokence
                        ctsl=cbp-(cbp*0.1)
                        print(f'DatetTime {str(cedf.index[-1])}, Spot:{spot}, ITM CE:{ce_buy_script}--> Buying Call @ {cbp}')
                    # elif order_status=='OPEN':
                    #     ## Buy at market price here
                    #     resp=exe_market_order('B','M','NFO',cscr,50,'Market order buy Triggred')
                    #     cbp=check_tradebook(cscr)
                    #     ce_buy_script=cscr
                    #     ce_buy_token=tokence
                    #     print(f'DatetTime {cedf.index[-1]}, Spot:{spot}, ITM CE:{ce_buy_script}--> Buying Call @ {cbp}')
                    else:
                        ce_buy_script=cscr
                        ce_buy_token=tokence
                        print('CE Order not Filled')
            elif ((order_status_ce=='OPEN') | (order_status_ce=='REJECTED')) & (now<datetime.time(15,0)):
                print(f'previous order is still open or got rejected  for {ce_buy_script}!! Please Check')
                
                    

        if (ceposition==True) & (order_status_ce=='COMPLETE'):
            if (cedf['Supertrend_y'].iloc[-1]==False) | (cedf['rsi_y'].iloc[-1]<50) | (now>=datetime.time(15,0)) | (cedf.Close[-1]<ctsl):
                ### Exit limit Order Here,but better to setup market order to make sure order will get executed.
                #current_ce_price=cedf.Close[-1]
                #resp_ce=exe_limt_order('S','M','NFO',ce_buy_script,50,current_ce_price,current_ce_price,' CE Limit order Sell Triggred')
                ## Following line will triger market Sell  order 
                resp_ce=exe_market_order('S','M','NFO',ce_buy_script,50,'CE Market order Sell Triggred')
                time.sleep(1)
                order_status=check_orderbook(ce_buy_script)
                if order_status=='COMPLETE':
                    ccp=check_tradebook(ce_buy_script)
                    ce_buy_script=''
                    ce_buy_token=''
                    print(f'DatetTime {str(cedf.index[-1])}, Spot:{spot}, ITM CE:{ce_buy_script}--> Covered Call @ {ccp}')
                else :
                    print(f'Not Able to Exit from Position {ce_buy_script}..Check Immediatly !!!!!')
                    ## exit at market price here 
                    # ccp=check_tradebook(cscr)
                    # ce_buy_script=''
                    # ce_buy_token=''
            else:
                print(f'In CE {ce_buy_script} position and no Cover triggered')
        
        if (peposition==False):
            #Before placing order checking if no other order for same script in open state.
            
            if ((order_status_pe=='COMPLETE') | (order_status_pe=='UNKNOWN')) & (now<datetime.time(15,0)):##Here Check if you already have position in ITM Call Option
                if (pedf['Supertrend_y'].iloc[-1]==True) & (pedf['Supertrend_x'].iloc[-1]==True)& (pedf['rsi_y'].iloc[-1]>50) & (pedf['rsi_y'].iloc[-1]>pedf['rsima_y'].iloc[-1]):
                    current_pe_price=pedf.Close[-1]
                    ## Place Limit/Market order here. if limit order not failed you will end up add multilple orders.so need to careful here                
                    resp_pe=exe_limt_order('B','M','NFO',pscr,50,current_pe_price,current_pe_price,'PE Limit order Buy Triggred')
                    ## Folloing line will triger market Buy order 
                    #resp_pe=exe_market_order('B','M','NFO',pscr,50,'PE Market order buy Triggred')
                    ## Check if oder is completed,wait for 1 sec before checking
                    time.sleep(2)
                    order_status=check_orderbook(pscr)
                    if order_status=='COMPLETE':
                        #get buy price from trade book.Folling function will return last price for the script
                        pbp=check_tradebook(pscr)
                        pe_buy_script=pscr
                        pe_buy_token=tokenpe
                        ptsl=pbp-(pbp*0.1)
                        print(f'DatetTime {str(pedf.index[-1])}, Spot:{spot}, ITM CE:{pe_buy_script}--> Buying Call @ {pbp}')
                    
                    else:
                        pe_buy_script=pscr
                        pe_buy_token=tokenpe
                        print(f'PE Order not Filled for {pe_buy_script}')
            elif ((order_status_pe=='OPEN') | (order_status_pe=='REJECTED')) & (now<datetime.time(15,0)):
                print(f'previous order is still open or got rejected  for {pe_buy_script}!! Please Check')
                
                    

        if (peposition==True) & (order_status_pe=='COMPLETE'):
            if (pedf['Supertrend_y'].iloc[-1]==False) | (pedf['rsi_y'].iloc[-1]<50) | (now>=datetime.time(15,0)) | (pedf.Close[-1]<ptsl):
                ### Exit limit Order Here,but better to setup market order to make sure order will get executed.
                #current_pe_price=pedf.Close[-1]
                #resp_pe=exe_limt_order('S','M','NFO',pe_buy_script,50,current_pe_price,current_pe_price,'PE Limit order Sell Triggred')
                ## Following line will triger market Buy order 
                resp_pe=exe_market_order('S','M','NFO',pe_buy_script,50,'PE Market order Sell Triggred')
                time.sleep(1)
                order_status=check_orderbook(pe_buy_script)
                if order_status=='COMPLETE':
                    pcp=check_tradebook(pe_buy_script)
                    pe_buy_script=''
                    pe_buy_token=''
                    print(f'DatetTime {str(pedf.index[-1])}, Spot:{spot}, ITM PE:{pe_buy_script}--> Covered Put @ {pcp}')
                else :
                    print(f'Not Able to Exit from Position {pe_buy_script}..Check Immediatly !!!!!')
                  
            else:
                print(f'In PE {pe_buy_script} position and no Cover triggered')


In [ ]:
datadf=resampler(datadf,'5min')

In [61]:
# resp=exe_limt_order('B','M','NFO','NIFTY17MAR22C16600',50,231,None,'Limit order Buy  Triggred')

In [62]:
resp

{'request_time': '18:40:57 14-03-2022',
 'stat': 'Ok',
 'norenordno': '22031400210413'}

In [22]:
mytoken=searchsymbol('NSE','Nifty 50')

In [44]:
type(mytoken)

str

In [23]:
ticker='ITC-EQ'
#ticker='NIFTY17MAR22P16650'
orderstatus=check_orderbook(ticker)
orderstatus

 order filled!! check position book


'COMPLETE'

In [24]:
check_position(ticker)

not in position


False

In [25]:
posret = api.get_positions()
positionsdf=pd.DataFrame(posret)
in_positions=positionsdf.loc[positionsdf.tsym=='NIFTY17MAR22C16600']


In [26]:
in_positions

,stat,uid,actid,exch,tsym,prd,token,dname,pp,ls,...,opensellamt,opensellavgprc,dayavgprc,netqty,netavgprc,upldprc,netupldprc,lp,urmtom,rpnl
0,Ok,FA54439,FA54439,NFO,NIFTY17MAR22C16600,M,50630,NIFTY 17MAR22 16600 CE,2,50,...,0.00,0.00,0.00,0,0.00,0.00,0.00,395.00,0.00,700.00


##### orderret = api.get_order_book()
orderdf=pd.DataFrame(orderret)
in_orderbook=orderdf.loc[orderdf.tsym=='NIFTY17MAR22C16600']
in_orderbook

In [28]:
orderdf.loc[orderdf.tsym=='NIFTY17MAR22C16600']['status']

4    COMPLETE
5    COMPLETE
Name: status, dtype: object

In [29]:
order1 = api.get_order_book()
order1df=pd.DataFrame(order1)
in_orderbook1=order1df.loc[order1df.tsym=='NIFTY17MAR22C16600']

In [30]:
in_orderbook1

,stat,norenordno,uid,actid,exch,tsym,qty,ordenttm,trantype,prctyp,...,norentm,exch_tm,exchordid,rqty,cancelqty,rejreason,rorgqty,dname,trgprc,rorgprc
4,Ok,22031400038408,FA54439,FA54439,NFO,NIFTY17MAR22C16600,50,1647232600,S,LMT,...,10:16:31 14-03-2022,14-03-2022 10:06:40,1000000031326925,50,NaN,NaN,50,NIFTY 17MAR22 16600 CE,245.00,240.00
5,Ok,22031400036193,FA54439,FA54439,NFO,NIFTY17MAR22C16600,50,1647231638,B,LMT,...,09:50:50 14-03-2022,14-03-2022 09:50:38,1000000029270545,50,NaN,NaN,50,NIFTY 17MAR22 16600 CE,NaN,230.10


In [31]:
tradebook = api.get_trade_book()
tradebookdf=pd.DataFrame(tradebook)
tradebookdf

,stat,norenordno,uid,actid,exch,prctyp,ret,prd,flid,fltm,...,fillshares,flqty,pp,ls,ti,prc,flprc,norentm,exch_tm,exchordid
0,Ok,22031400184893,FA54439,FA54439,NSE,MKT,DAY,C,30470381,14-03-2022 14:55:45,...,1,1,2,1,0.05,0.00,238.90,14:55:45 14-03-2022,14-03-2022 14:55:45,1100000020042308
1,Ok,22031400100901,FA54439,FA54439,NSE,MKT,DAY,C,28062224,14-03-2022 11:50:37,...,1,1,2,1,0.05,0.00,237.00,11:50:37 14-03-2022,14-03-2022 11:50:37,1100000011188161
2,Ok,22031400038408,FA54439,FA54439,NFO,LMT,DAY,M,2547077,14-03-2022 10:16:31,...,50,50,2,50,0.05,245.00,245.00,10:16:31 14-03-2022,14-03-2022 10:06:40,1000000031326925
3,Ok,22031400036193,FA54439,FA54439,NFO,LMT,DAY,M,1848778,14-03-2022 09:50:50,...,50,50,2,50,0.05,231.00,231.00,09:50:50 14-03-2022,14-03-2022 09:50:38,1000000029270545


In [32]:
check_orderbook('NIFTY17MAR22C16600')

 order filled!! check position book


'COMPLETE'

In [33]:
posret1 = api.get_positions()
positionsdf1=pd.DataFrame(posret1)
in_positions1=positionsdf1.loc[positionsdf1.tsym=='NIFTY17MAR22C16600']

In [34]:
## if net quantityis 0 then not in position other wise position is still open
int(in_positions1['netqty'].iloc[0])

0

In [35]:
# logoutme()

In [36]:
# api.get_security_info(exchange='NFO', token=tokenpe)

In [37]:
# lastBusDay = datetime.datetime.today()-datetime.timedelta(days=1)
# lastBusDay = lastBusDay.replace(hour=9, minute=16, second=0, microsecond=0)

In [38]:
# ret = api.get_time_price_series(exchange='NSE', token='26000', starttime=lastBusDay.timestamp(), interval=1)
# data=pd.DataFrame(ret)[::-1]

In [39]:
# datadf=data.copy()

In [40]:
# datadf['time']=pd.to_datetime(datadf['time'].astype(str), format='%d-%m-%Y %H:%M:%S')

In [41]:
# datadf.set_index('time',inplace=True)

In [42]:
# datadf=datadf.iloc[:,2:]
# datadf=datadf.astype(float)

In [43]:
# datadf5=resampler(datadf,'5min')
# datadf10=resampler(datadf,'10min')